In [1]:
import asyncpraw
from asyncpraw.models import MoreComments
import sys
import datetime
import json
import os
from pprint import pprint
import pandas as pd
import requests
import time

In [2]:
fileTitle = 'WSBtop{}year.json'
reddit = asyncpraw.Reddit(
    user_agent="Post Extraction",
    client_id="qCnN90vDQinbkA",
    client_secret="WtqnC08wNER7UAl8qayvCvxXM7jOFQ")

In [3]:
wsb = await reddit.subreddit("wallstreetbets")

In [ ]:
dic = {}

async for submission in wsb.top(time_filter='all',limit=10000):
    if submission.author is None:
        author_name = 'deleted'
    else:
        author_name = submission.author.name
    comments = await submission.comments()
    top_level_comments = []
    for comment in comments:
        if isinstance(comment,MoreComments):
            continue
        top_level_comments.append(comment.body)
    dic[submission.id] = {'title':submission.title,'self_text':submission.selftext,
                          'author_name':author_name,'upvotes':submission.ups,
                          'datetime':str(datetime.datetime.fromtimestamp(submission.created)),
                          'top_level_comments':top_level_comments}

fileTitle = 'WSBtop{}year.json'.format(str(len(dic)))
with open(fileTitle,'w+') as f:
    json.dump(dic,f)

In [66]:
base_url = 'https://www.reddit.com/'
app_ID = 'qCnN90vDQinbkA'
app_SECRET = 'WtqnC08wNER7UAl8qayvCvxXM7jOFQ'

auth = requests.auth.HTTPBasicAuth(app_ID, app_SECRET)
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'japooki',
        'password': 'KZFngdYaZ2gWfXG'}
# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']
# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [67]:
time_range_limits = [datetime.datetime(2020,1,1,0,0,0),
                         datetime.datetime(2021,1,25,2,30,0),
                         datetime.datetime(2021,2,4,21,0,0),
                         datetime.datetime.today()]
num_words_in_ranges = [0,0,0]
target_num_words_in_each_range = 100000

base_url = "https://oauth.reddit.com/"
df = pd.DataFrame(columns=['datetime','post_ID','score','title',
                           'num_words','num_chars',
                           'num_self_text_words','num_self_text_chars',
                           'self_text','comments','comment_scores',
                           'num_comment_words','num_comment_chars'])
resp = requests.get(base_url+'r/wallstreetbets/top?t=all',headers=headers)
resp = resp.json()
new_postID = resp['data']['children'][0]['data']['id']
epochUTC = resp['data']['children'][0]['data']['created_utc']
timestamp_str = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(epochUTC))
timestamp_datetime = datetime.datetime.strptime(timestamp_str,'%Y-%m-%d %H:%M:%S')
min_date_time = datetime.datetime.today()

while any(x < target_num_words_in_each_range for x in num_words_in_ranges):
    resp = requests.get(base_url+'r/wallstreetbets/top?after=t3_{}&t=all&limit=100'.format(new_postID),headers=headers)
    print('Requesting: {}'.format(base_url+'r/wallstreetbets/top?after=t3_{}&t=all'.format(new_postID)))
    previousPosts = resp.json()
    if previousPosts['data']['children'] == []:
        print('No children under request')
        sys.exit(0)
    for previousPost in previousPosts['data']['children']:
        post_ID = previousPost['data']['id']
        if post_ID in list(df.post_ID):
            print('continuing...')
            continue
        post_title = previousPost['data']['title']
        post_self_text = previousPost['data']['selftext']
        post_score = previousPost['data']['score']
        post_epoch = previousPost['data']['created_utc']
        post_timestamp_str = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(post_epoch))
        post_timestamp_datetime = datetime.datetime.strptime(post_timestamp_str,'%Y-%m-%d %H:%M:%S')
        for j in range(len(time_range_limits)-1):
            if post_timestamp_datetime > time_range_limits[j] and post_timestamp_datetime < time_range_limits[j+1]:
                range_lower_limit = time_range_limits[j]
                range_upper_limit = time_range_limits[j+1]
                df_tmp = df.loc[df.datetime > range_lower_limit,:]
                df_tmp = df_tmp.loc[df_tmp.datetime < range_upper_limit,:]
                num_words_in_this_range = sum(df_tmp.num_words)
                
        if num_words_in_this_range < target_num_words_in_each_range:
            resp = requests.get(base_url+'r/wallstreetbets/comments/{}?limit=500'.format(post_ID),headers=headers)
            resp = resp.json()
            comments = []
            comment_scores = []
            num_words = 0
            num_chars = 0
            for commentCollection in resp[1]['data']['children']:
                if commentCollection['kind'] == 'more':
                    continue
                comment = commentCollection['data']['body']
                comments.append(comment)
                comment_scores.append(commentCollection['data']['score'])
                num_words = num_words + len(comment.split())
                num_chars = num_chars + len(comment)
                if num_words > 1000:
                    break

            num_comment_words = num_words
            num_comment_chars = num_chars
            self_text_words = post_self_text.split()
            num_self_text_words = len(self_text_words)
            num_self_text_chars = len(post_self_text)
            title_words = post_title.split()
            num_words = num_words + num_self_text_words
            num_words = num_words + len(title_words)
            num_chars = num_chars + len(post_self_text)
            num_chars = num_chars + len(post_title)
            df = df.append({'datetime':post_timestamp_datetime,'post_ID':post_ID,'score':post_score,'title':post_title,
                            'num_words':num_words,'num_chars':num_chars,
                            'num_self_text_words':num_self_text_words,'num_self_text_chars':num_self_text_chars,
                            'self_text':post_self_text,'comments':comments,'comment_scores':comment_scores,
                            'num_comment_words':num_comment_words,'num_comment_chars':num_comment_chars},ignore_index=True)
            for j in range(len(time_range_limits)-1):
                df_tmp = df.loc[df.datetime > time_range_limits[j]]
                df_tmp = df_tmp.loc[df_tmp.datetime < time_range_limits[j+1]]
                num_words_in_ranges[j] = sum(df_tmp.num_words)
            print('After ({}), number of words in ranges: {}'.format(post_ID,num_words_in_ranges))
    new_postID = post_ID


Requesting: https://oauth.reddit.com/r/wallstreetbets/top?after=t3_l8rf4k&t=all
After (l6wu59), number of words in ranges: [0, 1014, 0]
After (l78uct), number of words in ranges: [0, 2254, 0]
After (l846a1), number of words in ranges: [0, 3336, 0]
After (l6x130), number of words in ranges: [0, 4438, 0]
After (l881ia), number of words in ranges: [0, 5556, 0]
After (l8c0u4), number of words in ranges: [0, 6056, 0]
After (l7feld), number of words in ranges: [0, 7855, 0]
After (l6ekdz), number of words in ranges: [0, 9058, 0]
After (lae6j0), number of words in ranges: [0, 10292, 0]
After (l6jobf), number of words in ranges: [0, 11322, 0]
After (lexy8t), number of words in ranges: [0, 11322, 1023]
After (l890i7), number of words in ranges: [0, 12436, 1023]
After (l90oq6), number of words in ranges: [0, 13776, 1023]
After (l79x17), number of words in ranges: [0, 14807, 1023]
After (l74tr1), number of words in ranges: [0, 15933, 1023]
After (l9kn3z), number of words in ranges: [0, 16942, 1023

After (i8yc0w), number of words in ranges: [14116, 100408, 30512]
After (kprwen), number of words in ranges: [15119, 100408, 30512]
After (gak2gf), number of words in ranges: [16146, 100408, 30512]
After (kqy5e4), number of words in ranges: [17159, 100408, 30512]
After (kwpviw), number of words in ranges: [18250, 100408, 30512]
After (lh0r8n), number of words in ranges: [18250, 100408, 31540]
After (fxw3uz), number of words in ranges: [19843, 100408, 31540]
After (ll68pc), number of words in ranges: [19843, 100408, 33349]
After (ldn46e), number of words in ranges: [19843, 100408, 34398]
Requesting: https://oauth.reddit.com/r/wallstreetbets/top?after=t3_l6l6b2&t=all
After (k3n4dx), number of words in ranges: [20826, 100408, 34398]
After (lkjppf), number of words in ranges: [20826, 100408, 35592]
After (g18kt2), number of words in ranges: [21837, 100408, 35592]
After (lfm36e), number of words in ranges: [21837, 100408, 36616]
After (j8zfc6), number of words in ranges: [22848, 100408, 366

After (fx8msd), number of words in ranges: [90685, 100408, 75482]
After (fxp9mh), number of words in ranges: [91697, 100408, 75482]
Requesting: https://oauth.reddit.com/r/wallstreetbets/top?after=t3_lbb59m&t=all
After (kq43kg), number of words in ranges: [92828, 100408, 75482]
After (itajte), number of words in ranges: [93837, 100408, 75482]
After (hwmtac), number of words in ranges: [94807, 100408, 75482]
After (glqgth), number of words in ranges: [95814, 100408, 75482]
After (f16r5m), number of words in ranges: [97285, 100408, 75482]
After (hqfn59), number of words in ranges: [98294, 100408, 75482]
After (e0togh), number of words in ranges: [98294, 100408, 75482]
After (f6ebcz), number of words in ranges: [99306, 100408, 75482]
After (g77r2w), number of words in ranges: [100335, 100408, 75482]
After (lj0slc), number of words in ranges: [100335, 100408, 76494]
After (lhj1so), number of words in ranges: [100335, 100408, 77791]
After (llj7oc), number of words in ranges: [100335, 100408,

SystemExit: 0

C:\Users\Colton\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [70]:
df.to_csv('100k_words_1-25_2-4.csv')

In [71]:
df

,datetime,post_ID,score,title,num_words,num_chars,num_self_text_words,num_self_text_chars,self_text,comments,comment_scores,num_comment_words,num_comment_chars
0,2021-01-28 13:40:34,l6wu59,303918,UPVOTE so everyone sees we got SUPPORT,1014,5822,0,0,,[This just makes me more eager to fuck them ov...,"[11729, 4355, 4201, 5548, 5733, 422, 7805, 990...",1007,5784
1,2021-01-28 21:06:23,l78uct,270378,GME YOLO update — Jan 28 2021,1240,7179,0,0,,[And once again!\n\nWhen i find myself in time...,"[10117, 27147, 22246, 25331, 7255, 17504, 1090...",1233,7150
2,2021-01-29 21:04:45,l846a1,243781,GME YOLO month-end update — Jan 2021,1082,5725,0,0,,[back up to 46mil. legend. dont waste your awa...,"[7881, 22743, 11384, 5996, 12723, 32627, 32030...",1075,5689
3,2021-01-28 13:49:11,l6x130,218587,CLASS ACTION AGAINST ROBINHOOD. Allowing peopl...,1102,6723,35,183,LEAVE ROBINHOOD. They dont deserve to make mon...,[The fucking audacity to call yourself Robin H...,"[23030, 13949, 8016, 11625, 1868, 1801, 23017,...",1015,6249
4,2021-01-29 23:40:59,l881ia,216551,It’s treason then,1118,6095,0,0,,[The real stonks are the friends we are making...,"[16628, 8161, 8706, 3886, 5153, 2322, 489, 105...",1115,6078
...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,2021-02-05 02:46:40,lcxjmx,19983,THE MODS ARE BACK!! SPREAD THE WORD!!,1015,6148,0,0,,"[Where’s zjz\n\nDaddy help me, Why is my relat...","[1392, 1405, 485, 157, 281, 74, 42, 184, 33, 3...",1008,6111
279,2019-10-10 17:56:43,dg1rxh,19872,"Aim high, Robinhood!",1005,5482,0,0,,[Lmao “where is you? Next to fruit ninja &amp;...,"[3746, 1233, 345, 562, 137, 193, 225, 150, 60,...",1002,5462
280,2019-09-17 21:14:29,d5ncec,19867,WeWork’s IPO is failing because the customers ...,1062,5704,0,0,,"[[deleted], just move the umbrella, Just throw...","[3377, 2640, 925, 1182, 566, 1161, 211, 377, 2...",1049,5624
281,2021-02-05 12:02:45,ld5rd9,19707,Evidence pointing to shorts did not cover pret...,3602,23536,2567,17842,"Long post ahead, but I encourage you to read t...",[This makes sense of that 43 million buy of mi...,"[2775, 1405, 991, 1141, 738, 305, 139, 114, 44...",1019,5599


In [50]:
num_words

1049

In [ ]:
datetime.datetime(2021,2,4,21,0,0)

In [ ]:
datetime.datetime.today()

In [18]:
print('abc: {}'.format([1,2,3]))

abc: [1, 2, 3]


In [21]:
[1,2,3].all()>0

AttributeError: 'list' object has no attribute 'all'

In [22]:
all(x>0 for x in [1,2,3])

True